In [1]:
from datetime import datetime, timedelta
import pandas as pd

In [2]:
import boto3
from boto3.dynamodb.conditions import Key

In [3]:
import json

In [4]:
# Given analysis window
analysis_window = '2020-07-25_Hour=15'

# Get the current datetime object
today_str = analysis_window[:10]
today_dt = datetime.strptime(today_str, '%Y-%m-%d')

In [5]:
# Initialize DynamoDB connection
table_name  = 'test_results'
index_name = 'analysis_date_index'

dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table(table_name)

# Store a list of dataframes with the sentiment label and scores for each stock and each day.
sentiment_df_list = []

# Past 7 days includes today.
for i in range(7):
    current_dt = today_dt - timedelta(days=i)
    
    # Use global secondary index to query data for a given day only.
    current_str = current_dt.strftime('%Y-%m-%d')
    response = table.query(
                IndexName=index_name,
                KeyConditionExpression=Key('analysis_date').eq(current_str)
                )
    
    # Columns: analysis_date, analysis_window, api_success_e_str, final_score, label, t_symb
    current_date_df = pd.DataFrame(response['Items'])

    # Get the date string for the most latest hour
    latest_hours_df = current_date_df.groupby('t_symb').agg({'analysis_window':'max'}).reset_index()

    # Fetch subset of data for the most latest hour of that day
    subset_hour_df = pd.DataFrame.merge(current_date_df, latest_hours_df, on=['t_symb','analysis_window'], how='inner')

    # Get the date, ticker symbol, api_success_est_time and predicted sentiment label and score
    current_day_sentiment_df = subset_hour_df[['analysis_date','t_symb', 'label', 'final_score','api_success_e_str']]
    
    # Add a categorical column to identify 1st (today) to 7th (furthest in the past) day 
    # using labels day-1 to day-7
    current_day_sentiment_df = current_day_sentiment_df.assign(day_label=f'day-{i+1}')
    
    sentiment_df_list.append(current_day_sentiment_df)

In [6]:
# Combine the list of dataframes into one long dataframe
sentiment_7_days_df = pd.concat(sentiment_df_list,axis=0)

In [7]:
# Rename label to sentiment label
sentiment_7_days_df = sentiment_7_days_df.rename(columns={'label':'sentiment_label'})

In [8]:
# Convert the date string to a "month/day" string
def gen_month_day_string(input_date_string):
    m_day = input_date_string[-5:]
    month, day = m_day.split('-')
    return f'{month}/{day}'

sentiment_7_days_df = sentiment_7_days_df.assign(month_day=sentiment_7_days_df
                                           .analysis_date
                                           .apply(gen_month_day_string))

# Chop off the 4 digit timezone info in last updated e.g. EDT-0400 to EDT
sentiment_7_days_df = sentiment_7_days_df.assign(last_updated = sentiment_7_days_df
                                           .api_success_e_str
                                           .map(lambda x: x[:-5]))

In [9]:
color_map = {'POSITIVE':'#1a9641', 'NEGATIVE':'#d7191c', 'UNCERTAIN':'#607d8b'}

In [10]:
sentiment_7_days_df = sentiment_7_days_df.assign(hex_color=sentiment_7_days_df
                                                                .sentiment_label
                                                                .map(color_map))

In [11]:
sentiment_7_days_df.shape

(35, 9)

In [12]:
sentiment_7_days_df.head(10)

,analysis_date,t_symb,sentiment_label,final_score,api_success_e_str,day_label,month_day,last_updated,hex_color
0,2020-07-25,AMZN,NEGATIVE,-0.90352811,2020-07-25 15:42:53 EDT-0400,day-1,07/25,2020-07-25 15:42:53 EDT,#d7191c
1,2020-07-25,GOOG,NEGATIVE,-0.97650835,2020-07-25 15:43:34 EDT-0400,day-1,07/25,2020-07-25 15:43:34 EDT,#d7191c
2,2020-07-25,NFLX,NEGATIVE,-0.97371604,2020-07-25 15:43:17 EDT-0400,day-1,07/25,2020-07-25 15:43:17 EDT,#d7191c
3,2020-07-25,FB,NEGATIVE,-0.99109565,2020-07-25 15:42:42 EDT-0400,day-1,07/25,2020-07-25 15:42:42 EDT,#d7191c
4,2020-07-25,AAPL,NEGATIVE,-0.81748209,2020-07-25 15:43:06 EDT-0400,day-1,07/25,2020-07-25 15:43:06 EDT,#d7191c
0,2020-07-24,NFLX,NEGATIVE,-0.99483493,2020-07-25 15:43:17 EDT-0400,day-2,07/24,2020-07-25 15:43:17 EDT,#d7191c
1,2020-07-24,AAPL,UNCERTAIN,-0.01576584,2020-07-25 15:43:06 EDT-0400,day-2,07/24,2020-07-25 15:43:06 EDT,#607d8b
2,2020-07-24,FB,UNCERTAIN,-0.16393157,2020-07-25 15:42:42 EDT-0400,day-2,07/24,2020-07-25 15:42:42 EDT,#607d8b
3,2020-07-24,GOOG,UNCERTAIN,-0.40299231,2020-07-25 15:43:34 EDT-0400,day-2,07/24,2020-07-25 15:43:34 EDT,#607d8b
4,2020-07-24,AMZN,NEGATIVE,-0.76449714,2020-07-25 15:42:53 EDT-0400,day-2,07/24,2020-07-25 15:42:53 EDT,#d7191c


### Store on DynamoDB

Store as JSON file which should be inserted twice into the table.The first insert should overwrite the data on the latest key, so that latest data can be easily pulled. The second key is just the analysis window to keep track of previous public results.

In [13]:
# From spark_script.py
class DynamoDBHelper:
    def __init__(self):
        self.dynamodb_conn = boto3.resource('dynamodb', region_name='us-west-2')

    def read_table(self, table_name, target_analysis_window):
        table = self.dynamodb_conn.Table(table_name)
        response = table.query(KeyConditionExpression=Key('analysis_window')
                               .eq(target_analysis_window)
                               )

        # This is a list of dictionaries where
        # the keys of each dictionary is a key/column in the DynamoDB table
        news_data = response['Items']

        return news_data

    def write_table(self, table_name, data_dict_list):

        results_table = self.dynamodb_conn.Table(table_name)
        # Use batch writer to automatically handle buffering and sending items in batches
        # https://boto3.amazonaws.com/v1/documentation/api/latest/guide/dynamodb.html
        with results_table.batch_writer() as batch:
            for data_row in data_dict_list:
                batch.put_item(
                    Item=data_row
                )

    def write_item(self, table_name, item):
        target_table = self.dynamodb_conn.Table(table_name)
        response = target_table.put_item(Item=item)
        response_code = response['ResponseMetadata']['HTTPStatusCode']
                


In [15]:
data_dict_list = sentiment_7_days_df.to_dict('records')

In [16]:
dynamo = DynamoDBHelper()

In [17]:
dynamo.write_table(table_name='public_results', data_dict_list=data_dict_list)

### Write as nested JSON

In [15]:
# Create nested dictionary from pandas dataframe with t_symb as first level key 
# and day_label as second level key
# E.g. FB.day-1.sentiment_label, ..., AMZN.day-6.month_day
# https://stackoverflow.com/questions/41998624/how-to-convert-pandas-dataframe-to-nested-dictionary
nested_7_days = (sentiment_7_days_df.groupby('t_symb')[['day_label','sentiment_label',
                                                        'analysis_date','month_day','hex_color',
                                                        'last_updated']]
                                   .apply(lambda x: x.set_index('day_label').to_dict(orient='index'))
                                   .to_dict()
                )

In [16]:
json_string = json.dumps(nested_7_days,indent=4)

In [17]:
with open("test_7_days.json", "w") as json_file:
    print(json_string, file=json_file)